# Mutual Interactors
Mutual Interactors is a machine learning algorithm for node set expansion in large networks. The algorithm is motivated by the structure of disease-associated proteins, drug targets and protein functions in molecular networks, and can be used to predict molecular phenotypes in silico. For a detailed description of the algorithm, please see our paper.

In this notebook, we will walk through how we train a Mutual Interactors model to predict novel disease protein associations. We use a PPI network and a large set of disease-protein associations to train the model.

Although this notebook uses a PPI network and disease protein associations, it can easily be retrofitted to work with any network and any node set type.

In [1]:
!git clone https://github.com/seyuboglu/milieu.git

Cloning into 'milieu'...
remote: Enumerating objects: 4450, done.
remote: Counting objects: 100% (1489/1489), done.
remote: Compressing objects: 100% (760/760), done.
remote: Total 4450 (delta 775), reused 1169 (delta 614), pack-reused 2961 (from 1)
Receiving objects: 100% (4450/4450), 182.33 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (2095/2095), done.
Updating files: 100% (1142/1142), done.


In [37]:
%cd milieu
# os.getcwd()

/content/milieu/milieu


In [3]:
!pip install torch goatools parse ndex2 cyjupyter
%load_ext autoreload
%autoreload 2

import os

import networkx as nx

from milieu.data.network import Network
from milieu.data.associations import load_diseases
from milieu.util.util import load_mapping
from milieu.milieu import MilieuDataset, Milieu
from milieu.paper.figures.network_vis import show_network

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
   ━━

## Load a Network
To use Mutual Interactors we need a network!

We'll use the human protein-protein interaction network compiled by Menche et al.[1]. The network consists of 342,353 interactions between 21,557 proteins. Se In data/networks, you can find this network bio-pathways-network.txt. See methods for a more detailed description of the network.

We use the class milieu.data.network.Network to load and represent networks. The constructor accepts a path to an edge list.

In [4]:
# Patch the __init__ method for the Network class
import os
import logging
import random

import numpy as np
import pandas as pd
import networkx as nx

def __patched_init__(self, network_path, remove_edges=0, remove_nodes=0):
        """
        Patched __init__ method for the Network class.
        Replaces nx.from_numpy_matrix with nx.from_numpy_array.

        Load a protein-proetin interaction network from an adjacency list.
        args:
            network_path (string)
            remove_edges (double) fraction between 0 and 1 inclusive indicating
            fraction of edges to randomly remove
            remove_nodes (double) fraction between 0 and 1 inclusive indicating
            fraction of nodes to randomly remove
        """
        # map protein entrez ids to node index
        node_names = set()
        edges = []
        with open(network_path) as network_file:
            for line in network_file:
                if remove_edges > 0 and random.random() < remove_edges:
                    continue
                p1, p2 = [int(a) for a in line.split()]
                node_names.add(p1)
                node_names.add(p2)
                edges.append((p1, p2))
        if remove_nodes > 0:
            assert(remove_nodes < 1)
            node_names = random.sample(node_names, 1 - remove_nodes)

        self.name_to_node = {p: n for n, p in enumerate(node_names)}
        self.node_to_name = {n: p for p, n in self.name_to_node.items()}

        # build adjacency matrix
        self.adj_matrix = np.zeros((len(self.name_to_node),
                                    len(self.name_to_node)))
        for p1, p2 in edges:
            n1, n2 = self.name_to_node[p1], self.name_to_node[p2]
            self.adj_matrix[n1, n2] = 1
            self.adj_matrix[n2, n1] = 1

        # self.nx = nx.from_numpy_matrix(self.adj_matrix)
        self.nx = nx.from_numpy_array(self.adj_matrix)


# Apply the patch
Network.__init__ = __patched_init__

# Now you can use the Network class as before
network = Network("data/networks/species_9606/bio-pathways/network.txt")

## Build the Milieu Model
The Mutual Interactors is parameterized by a few important hyperparameters.

We find that learning rate parameter (i.e. optim_args/lr in the nested dictionary below) can have significant impact on performance. The optimal value varies substantially between networks and applications, so we recommend tuning it.

If you have a GPU available, setting cuda to True and specifying an available device should speed up training considerably. That being said, training Mutual Interactors is usually tractable on CPU for networks with
.

In [5]:
# Patch the build_model method for the Milieu class
# import os
import json
import logging
from shutil import copyfile
from collections import defaultdict
from copy import deepcopy

import numpy as np
import networkx as nx
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from scipy.sparse import coo_matrix, csr_matrix
import parse

from milieu.paper.methods.method import DPPMethod
from milieu.data.associations import load_node_sets
from milieu.util.metrics import compute_metrics
from milieu.util.util import set_logger, load_mapping

def _patched_build_model(self):
        """
        Initialize the variables and parameters of the Milieu model.
        See Methods, Equation (2) for corresponding mathematical definition.
        """
        # degree vector, (D^{-0.5} in Equation (2))
        degree = np.sum(self.adj_matrix, axis=1, dtype=float)
        inv_sqrt_degree = np.power(degree, -0.5)
        inv_sqrt_degree = torch.tensor(inv_sqrt_degree, dtype=torch.float)

        # adjacency matrix of network, (A in Equation (2))
        adj_matrix = torch.tensor(self.adj_matrix, dtype=torch.float)

        # precompute the symmetric normalized adj matrix, used on the left of Equation (2)
        adj_matrix_left = torch.mul(torch.mul(inv_sqrt_degree.view(1, -1),
                                              adj_matrix),
                                    inv_sqrt_degree.view(-1, 1))

        # precompute the normalized adj matrix, used on the right of Equation (2)
        adj_matrix_right = torch.mul(inv_sqrt_degree.view(1, -1),
                                     adj_matrix)
        self.register_buffer("adj_matrix_right", adj_matrix_right)
        self.register_buffer("adj_matrix_left", adj_matrix_left)

        # milieu weight vector, ('W' in Equation (2))
        self.milieu_weights = nn.Parameter(torch.ones(1, 1, adj_matrix.shape[0],
                                           dtype=torch.float,
                                           requires_grad=True))

        # scaling parameter, ('a' in in Equation (2))
        self.scale = nn.Linear(1, 1)

        # the bias parameter, ('b' in Equation (2))
        self.bias = nn.Parameter(torch.ones(size=(1,),
                                            dtype=torch.float,
                                            requires_grad=True))


# Apply the patch
Milieu._build_model = _patched_build_model

In [6]:
params = {
    "cuda": False,
    "device": 2,

    "batch_size": 200,
    "num_workers": 4,
    "num_epochs": 10,

    "optim_class": "Adam",
    "optim_args": {
        "lr": 0.01,
        "weight_decay": 0.0
    },

    "metric_configs": [
        {
            "name": "recall_at_25",
            "fn": "batch_recall_at",
            "args": {"k":25}
        }
    ]
}

We've implemented the Mutual Interactors model in a self-contained class milieu.milieu.Milieu. This class contains methods for training the model Milieu.train_model, evaluating the model on a test set Milieu.score and predicting node set expansions Milieu.expand.

The constructor accepts the network and the dictionary of params we defined above.

In [7]:
milieu = Milieu(network, params)

INFO:root:Milieu
INFO:root:Setting parameters...
INFO:root:Building model...
INFO:root:Building optimizer...
INFO:root:Done.


## Train the Model
Mutual Interactors is trained on a dataset of groups of nodes known to be associated with one another in some way.
In this example, we use sets of proteins associated with the same disease. Our disease-protein associations come from disgenet and are found at data/disease_associations/disgenet-associations.csv.

We load the disease-protein associations with milieu.data.associations.load_diseases which returns a list of milieu.data.associations.NodeSet. Each NodeSet represents the set of proteins associated with on disease.

To evaluate the model as we train it, we'll split the set of diseases into train set and a validation set. Next, we'll create a milieu.milieu.MilieuDataset for each. A MilieuDataset is simply a PyTorch dataset that creates training examples for the Mutual Interactors momdel.

In [8]:
node_sets = list(load_diseases("data/associations/disgenet/associations.csv", exclude_splits=["none"]).values())
train_node_sets = node_sets[:int(len(node_sets)* 0.9)]
valid_node_sets = node_sets[int(len(node_sets)* 0.9):]
train_dataset = MilieuDataset(network, node_sets=train_node_sets)
valid_dataset = MilieuDataset(network, node_sets=valid_node_sets)

In [9]:
milieu.train_model(train_dataset, valid_dataset)

INFO:root:Starting training for 10 epoch(s)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO:root:Epoch 1 of 10
INFO:root:Training
100%|██████████| 9/9 [00:58<00:00,  6.48s/it, loss=1.405]
INFO:root:Validation
  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader runnin

([{'recall_at_25': 0.006091663764077557},
  {'recall_at_25': 0.059985388050820466},
  {'recall_at_25': 0.06066606852165254},
  {'recall_at_25': 0.06508019929380804},
  {'recall_at_25': 0.05829975091137153},
  {'recall_at_25': 0.06253906905967097},
  {'recall_at_25': 0.0612237380727925},
  {'recall_at_25': 0.07128082395445749},
  {'recall_at_25': 0.07040908104168589},
  {'recall_at_25': 0.0709470499921994}],
 [defaultdict(list, {'recall_at_25': [0.04710632773819586]}),
  defaultdict(list, {'recall_at_25': [0.056331763474620614]}),
  defaultdict(list, {'recall_at_25': [0.06323854368967151]}),
  defaultdict(list, {'recall_at_25': [0.030811247157401005]}),
  defaultdict(list, {'recall_at_25': [0.06288809031724012]}),
  defaultdict(list, {'recall_at_25': [0.048472360972360976]}),
  defaultdict(list, {'recall_at_25': [0.055701502404799104]}),
  defaultdict(list, {'recall_at_25': [0.05781070433326072]}),
  defaultdict(list, {'recall_at_25': [0.06087483596882092]}),
  defaultdict(list, {'recal

##Predict Novel Associations
Now that we've got a trained Mutual Interactors model, we can use it to expand some node sets!

In particular, here we are going to use it to predict which proetins are associated with Tracheomalacia, a condition characterized by flaccidity of the supporting tracheal cartilage.

To do so, we specify the set of proteins associated with Tracheomalacia using GenBank IDs.

In [10]:
# Specify a set of proteins by their GenBank IDs
# For example, we use the proteins associated with Tracheomalacia
# Swap out these GenBank IDs for another set of proteins!
tracheomalacia_proteins = ['COL2A1', 'HRAS', 'DCHS1', 'SNRPB', 'ORC4', 'LTBP4',
                           'FLNB', 'PRRX1', 'RAB3GAP2', 'FGFR2','TRIM2']

In [11]:
# Convert genbank ids to entrez ids, since our network uses entrez ids
genbank_to_entrez = load_mapping("data/protein_attrs/genbank_to_entrez.txt",
                                 b_transform=int, delimiter='\t')
tracheomalacia_entrez = [genbank_to_entrez[protein] for protein in tracheomalacia_proteins]

In [12]:
# Expand the set of proteins using our trained model!
# Change the number of predicted proteins using the top_k parameter
predicted_entrez = milieu.expand(node_names=tracheomalacia_entrez, top_k=5)
predicted_entrez = list(zip(*predicted_entrez))[0]

Using the function milieu.paper.figures.network_vis.show_network we can generate a Cytoscape visualization of the predictions!

In [26]:
from google.colab import output
output.enable_custom_widget_manager()

In [27]:
 # Patch the show_network and get_network functiosn in milieu.paper.figures.network_vis
import json
import os
import random

import ndex2.client as nc
from cyjupyter import Cytoscape
import networkx as nx
import numpy as np

from milieu.util.util import load_mapping
from milieu.util.util import ensure_dir_exists

def patched_show_network(network, seed_proteins, pred_proteins,
                 id_format="genbank", style=None, show_seed_mi=True,
                 model=None, excluded_interactions=[], save_path=None,
                 size_limit=200):
    """
    Generate a cytoscape jupyter visualization for the induced subgraph of seed_proteins,
    pred_proteins and the mutual interactors between them.
    """
    if id_format == "genbank":
        genbank_to_entrez = load_mapping("data/protein_attrs/genbank_to_entrez.txt",
                                         b_transform=int, delimiter='\t')
        seed_proteins = [genbank_to_entrez[protein]
                         for protein in seed_proteins if protein in genbank_to_entrez]
        pred_proteins = [genbank_to_entrez[protein]
                         for protein in pred_proteins if protein in genbank_to_entrez]
        seed_nodes = network.get_nodes(seed_proteins)
        pred_nodes = network.get_nodes(pred_proteins)

    elif id_format == "entrez":
        seed_nodes = network.get_nodes(seed_proteins)
        pred_nodes = network.get_nodes(pred_proteins)
    else:
        raise ValueError("id_format is not recognized.")

    cyjs_network = patched_get_network(network, seed_nodes, pred_nodes, model,
                               show_seed_mi, excluded_interactions, size_limit)
    # Unique ID for a network entry in NDEx
    uuid = 'f28356ce-362d-11e5-8ac5-06603eb7f303'

    # NDEx public server URL
    ndex_url = 'http://public.ndexbio.org/'

    # Create an instance of NDEx client
    #ndex = nc.Ndex2(ndex_url)

    # Download the network in CX format
    #response = ndex.get_network_as_cx_stream(uuid)

    # Store the data in a Python object
    #cx = response.json()

    if style is None:
        style = [
            {
                "selector": "node",
                "css": {
                    "content": "data(genbank)",
                    "border-color" : "rgb(256,256,256)",
                    "border-opacity" : 1.0,
                    "border-width" : 2,

                },
            },
            {
                "selector": "node[role = 'seed']",
                "css": {
                    "background-color": "#f53e37",
                    "width": 20,
                    "height": 20
                },
            },
            {
                "selector": "node[role = 'pred']",
                "css": {
                    "background-color": "#ff9529",
                    "width": 20,
                    "height": 20
                },
            },
            {
                "selector": "node[role = 'mutual_interactor']",
                "css": {
                    "background-color": "#6599d1",
                    "width": 20,
                    "height": 20
                },
            }
        ]

    if save_path is not None:
        with open(save_path, 'w') as f:
            json.dump(cyjs_network, f, indent=4)

    cytoscape = Cytoscape(data=cyjs_network, visual_style=style)
    return cytoscape

def patched_get_network(network, seed_nodes, pred_nodes, model=None,
                show_seed_mi=True, excluded_interactions=[], size_limit=200):
    """ Get the disease subgraph of
    Args:
        disease: (Disease) A disease object
    """
    entrez_to_genbank = load_mapping("data/protein_attrs/genbank_to_entrez.txt",
                                     b_transform=int, delimiter='\t', reverse=True)
    nodes = {}

    def add_node(node, role="seed"):
        if node not in nodes:
            if model is not None and role == "mutual_interactor":
                weight = float(model.milieu_weights[0, 0, node] / np.sqrt(network.nx.degree(node)))
            else:
                weight = 1.0
            nodes[node] = {
                "data": {
                    "role": role,
                    "id": str(node),
                    "entrez": str(network.get_name(node)),
                    "genbank": entrez_to_genbank.get(network.get_name(node), ""),
                    "normalized_milieu_weight": weight
                }
            }

    # add seed nodes
    for seed_node in seed_nodes:
        add_node(seed_node, role="seed")

    # get seed node neighbors
    seed_node_to_nbrs = {node: set(network.nx.neighbors(node))
                         for node in seed_nodes}
    # get mutual interactors between preds and seed
    for pred_node in pred_nodes:
        add_node(pred_node, role="pred")
    for pred_node in pred_nodes:
        pred_nbrs = set(network.nx.neighbors(pred_node))
        for seed_node in seed_nodes:
            seed_nbrs = seed_node_to_nbrs[seed_node]
            common_nbrs = seed_nbrs & pred_nbrs
            for common_nbr in common_nbrs:
                add_node(common_nbr, role="mutual_interactor")

    # the set of nodes intermediate between nodes in the
    if show_seed_mi:
        for a, node_a in enumerate(seed_nodes):
            for b, node_b in enumerate(seed_nodes):
                # avoid repeat pairs
                if a >= b:
                    continue
                common_nbrs = seed_node_to_nbrs[node_a] & seed_node_to_nbrs[node_b]
                for common_nbr in common_nbrs:
                    add_node(common_nbr, role="mutual_interactor")

    if size_limit is not None:
        if size_limit < len(seed_nodes) + len(pred_nodes):
            raise ValueError(f"size_limit ({size_limit}) must be at least as large as the total number" +
                             f"of seed and predicted nodes ({len(seed_nodes) + len(pred_nodes)}).")
        while len(nodes) > size_limit:
            node = random.choice(list(nodes.keys()))
            node_data = nodes[node]["data"]
            if node_data["role"] == "mutual_interactor":
                del nodes[node]

    # get induced subgraph
    subgraph = nx.Graph(network.nx.subgraph(nodes.keys()))

    # subgraph.remove_edges_from(subgraph.selfloop_edges())
    subgraph.remove_edges_from(list(nx.selfloop_edges(subgraph)))  # Use nx.selfloop_edges to get self-loop edges

    edges = []
    for edge in subgraph.edges():
        if (nodes[edge[0]]["data"]["role"],
            nodes[edge[1]]["data"]["role"]) in excluded_interactions:
            continue
        edges.append({
            "data": {
                "source": str(edge[0]),
                "target": str(edge[1]),
                "roles": f'{nodes[edge[0]]["data"]["role"]}-{nodes[edge[1]]["data"]["role"]}'
            }
        })

    return {"elements": {"nodes": list(nodes.values()), "edges": edges}}

In [33]:
# Generate a network visualization with cytoscape
# Note: it is recommended to limit the size of the visualization to ~250 nodes
cy_vis = patched_show_network(network, tracheomalacia_entrez, predicted_entrez, id_format="entrez",
                      model=milieu,
                      show_seed_mi=True, excluded_interactions=[("mutual_interactor", "mutual_interactor")],
                      size_limit=250)

In [32]:
# Show the visualization!
# Red nodes are the seed nodes fed to the momdel.
# Orange nodes are predicted nodes. Blue nodes are the interactors between them.
cy_vis

Cytoscape(data={'elements': {'nodes': [{'data': {'role': 'seed', 'id': '925', 'entrez': '1280', 'genbank': 'CO…

# Milieu model for C. elegans

In [ ]:
# Load data
network = Network("data/networks/species_9606/bio-pathways/network.txt")

In [ ]:
# Patch the __init__ method for the Network class
import os
import logging
import random

import numpy as np
import pandas as pd
import networkx as nx

class PULoss(nn.Module):
    def __init__(self, positive_weight=1.0, unlabeled_weight=0.1):
        """
        PU Learning loss function for Mutual Interactors model.

        Args:
        - positive_weight (float): Weight for known positives.
        - unlabeled_weight (float): Initial weight for unlabeled nodes.
        """
        super(PULoss, self).__init__()
        self.positive_weight = positive_weight
        self.unlabeled_weight = unlabeled_weight

    def forward(self, outputs, labels, unlabeled_mask):
        """
        Compute PU learning loss.

        Args:
        - outputs (Tensor): Model probability outputs (logits).
        - labels (Tensor): Binary tensor (1 for positive nodes, 0 for others).
        - unlabeled_mask (Tensor): Binary tensor (1 for unlabeled nodes, 0 for positives).

        Returns:
        - loss (Tensor): Weighted binary cross-entropy loss.
        """
        pos_loss = -self.positive_weight * labels * torch.log(torch.sigmoid(outputs) + 1e-8)
        w_j = self.unlabeled_weight  # Can be refined dynamically
        unlabeled_loss = -w_j * unlabeled_mask * torch.log(1 - torch.sigmoid(outputs) + 1e-8)

        loss = pos_loss + unlabeled_loss
        return loss.mean()

class MilieuWorm(nn.Module):
    def __init__(self, ...):
        super(Milieu, self).__init__()
        self.loss_fn = PULoss(positive_weight=1.0, unlabeled_weight=0.5)  # Replace BCE loss

    def loss(self, outputs, targets, unlabeled_mask):
        return self.loss_fn(outputs, targets, unlabeled_mask)

    # def _build_model(self):
    #     """
    #     Initialize the variables and parameters of the Milieu model.
    #     See Methods, Equation (2) for corresponding mathematical definition.
    #     """
    #     # degree vector, (D^{-0.5} in Equation (2))
    #     degree = np.sum(self.adj_matrix, axis=1, dtype=float)
    #     inv_sqrt_degree = np.power(degree, -0.5)
    #     inv_sqrt_degree = torch.tensor(inv_sqrt_degree, dtype=torch.float)

    #     # adjacency matrix of network, (A in Equation (2))
    #     adj_matrix = torch.tensor(self.adj_matrix, dtype=torch.float)

    #     # precompute the symmetric normalized adj matrix, used on the left of Equation (2)
    #     adj_matrix_left = torch.mul(torch.mul(inv_sqrt_degree.view(1, -1),
    #                                           adj_matrix),
    #                                 inv_sqrt_degree.view(-1, 1))

    #     # precompute the normalized adj matrix, used on the right of Equation (2)
    #     adj_matrix_right = torch.mul(inv_sqrt_degree.view(1, -1),
    #                                  adj_matrix)
    #     self.register_buffer("adj_matrix_right", adj_matrix_right)
    #     self.register_buffer("adj_matrix_left", adj_matrix_left)

    #     # milieu weight vector, ('W' in Equation (2))
    #     self.milieu_weights = nn.Parameter(torch.ones(1, 1, adj_matrix.shape[0],
    #                                        dtype=torch.float,
    #                                        requires_grad=True))

    #     # scaling parameter, ('a' in in Equation (2))
    #     self.scale = nn.Linear(1, 1)

    #     # the bias parameter, ('b' in Equation (2))
    #     self.bias = nn.Parameter(torch.ones(size=(1,),
    #                                         dtype=torch.float,
    #                                         requires_grad=True))

